<a href="https://colab.research.google.com/github/showmik121/Genetivie_AI/blob/main/Text_summrizetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text summerization

step->

1.pakage and libaries install
2.dataset_load
3.tokenizer
4.Map->use ->transformation
5.Model load->parametter
6.Model Training
7.Evalution
8.Save model
9.check

In [ ]:
pip install --upgrade transformers[sentencepiece] datasets sacrebleu rouge_score py7zr --no-cache-dir

**Breakdown command**

---



1.   pip install --upgrade



Ensures that the latest version of the packages is installed.

2.   transformers[sentencepiece]


Installs the Hugging Face transformers library.
The [sentencepiece] extra is needed for models like T5, BART, and MarianMT, which use SentencePiece tokenization.
3.   datasets

Installs Hugging Face’s datasets library for loading and processing datasets like IMDB, SQuAD, etc.
4.sacrebleu

Used for BLEU score evaluation in machine translation tasks.
5. rouge_score

Used to calculate ROUGE scores for summarization tasks.
6.   py7zr

Supports .7z compression, which some datasets use
7.   --no-cache-dir


Prevents storing downloaded packages in cache (useful if storage is limited).




In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Updates the Hugging Face accelerate library, which optimizes model training and inference on GPUs/TPUs.
Useful when working with distributed training.

If models are not using GPUs efficiently, a fresh accelerate install may help

In [ ]:
pip install datasets

In [ ]:
pip install evaluate

used for evaluating machine learning models in NLP and other domains. Easy integration with datasets and transformers for NLP tasks.

In [ ]:
!pip install --upgrade datasets evaluate

# Import

In [ ]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
import evaluate

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

nltk (Natural Language Toolkit) is used for NLP tasks like tokenization.

**AutoModelForSeq2SeqLM is used for text generation tasks** **bold text**

Used in Summarization, Translation, and Paraphrasing

nltk.download("punkt")-> Splits a paragraph into sentences.

In [ ]:
import torch

# Pre trained model

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
#

# Data set- 1.id  2.dialoge  3.summary

In [ ]:
!pip install py7zr

In [ ]:
dataset_samsum=load_dataset('samsum')
dataset_samsum

In [ ]:
dataset_samsum['train']['dialogue'][0]

In [ ]:
dataset_samsum['train'][0]['summary']

In [ ]:
split_lengths=[len(dataset_samsum[split]) for split in dataset_samsum]

Order of values: [train_size, validation_size, test_size]

In [ ]:
print(f"Split length: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")



1.   split_lengths gives dataset size per split.
2.   dataset['train'].column_names lists dataset features



In [ ]:
print(dataset_samsum['test'][0]['dialogue'])
print(dataset_samsum['test'][0]['summary'])

# Tokenization-**label data**

In [ ]:
def convert_examples_to_feature(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)


  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
  }

(truncation=True) Prevents overflow errors when input text is longer than the model’s max length

# Transformation-**mapping**

In [ ]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_feature,batched=True)

Batch processing

In [ ]:
dataset_samsum_pt['train']['input_ids'][1]

In [ ]:
dataset_samsum_pt['train']['attention_mask'][1]

**data collator**

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2aeq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

DataCollatorForSeq2Seq-Ensures correct label padding

Ensures correct input format for Seq2Seq models like T5, BART, Pegasus.

Helps when using Trainer() for model training.

# Tranning

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2aeq_data_collator,
                  train_dataset=dataset_samsum_pt['train'],
                  eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
trainer.train()

# Evaluation

In [ ]:
def generate_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16,
                                column_text="article",
                                column_summary="highlights",
                                device=None): # Change 1: Remove device=device and add device=None
    if device is None: # Change 2: Check if device is provided and assign if not
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

        input_data = tokenizer(article_batch, max_length=1024, truncation=True,
                            padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=input_data["input_ids"].to(device),
                                  attention_mask=input_data["attention_mask"].to(device),
                                  length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
                            for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metrix=evaluate.load('rouge')

In [ ]:

# calculate metrix on test dataset
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metrix, model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
)
rouge_dict=dict((rn,score[rn]) for rn in rouge_names)

# create data frame on display
pd.DataFrame(rouge_dict,index=[f'pegasus'])

# Save Model

In [ ]:
# save model
model_pegasus.save_pretrined("pegasus-samsum")

In [ ]:
# save tokenizer
tokenizer.save_pretrained("tokenizer")

# Load

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
# prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

In [ ]:
sample_test=s=dataset_samsum["test"][0]["dialogue"]
reference=dataset_samsum["test"][0]["summary"]

In [ ]:
pipe=pipeline("summarization",model="pegasus-samsum",tokenizer=tokenizer)

In [ ]:
print("dialogue")
print(sample_test)

print("\nReference Summary")
print(reference)

print("\nModel Summary")
print(pipe(sample_test, **gen_kwargs)[0]["summary_text"])